### Book Rental Recommendation

Before reading data from a csv file, you need to download the “BX-Book-Ratings.csv”, “BX-Books.csv”, “BX-Users.csv”, and “Recommend.csv” datasets from the resource section and upload them into the Lab. We will use the Up arrow icon, which is shown on the left side under the View icon. Click on the Up arrow icon and upload the file from wherever it was downloaded into your system.

The objective is to recommend books to a user based on their purchase history and the behavior of other users.

Dataset Description
BX-Users:

* user_id - These have been anonymized and mapped to integers
* Location - Demographic data is provided
* Age - Demographic data is provided
        If available. Otherwise, these fields contain NULL-values.

BX-Books:

* isbn - Books are identified by their respective ISBNs. Invalid ISBNs have already been removed from the dataset.
* book_title
* book_author
* year_of_publication
* publisher

BX-Book-Ratings:
* user_id
* isbn
* rating - Ratings (Book-Rating) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

### Coding Start

In [1]:
#Import required libraries

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Read the data - BX-Users

To map byte values directly to the first 256 Unicode code points, use the "Latin-1" encoding. This is the closest equivalent Python 3 offers to the permissive Python 2 text handling model.

In [2]:
# read dataset
df_user = pd.read_csv('BX-Users.csv', encoding='latin-1')

In [3]:
# Preview the information of first 5 rows of dataset.

df_user.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:
# Checking for Null Values

df_user.isnull().any()

user_id     False
Location     True
Age          True
dtype: bool

### Dropping the null values

In [5]:
df_user1 = df_user.dropna()

In [6]:
df_user1.isnull().any()

user_id     False
Location    False
Age         False
dtype: bool

### Read the data - BX-Books

To map byte values directly to the first 256 Unicode code points, use the "Latin-1" encoding. This is the closest equivalent Python 3 offers to the permissive Python 2 text handling model.

In [7]:
# read dataset
df_books = pd.read_csv('BX-Books.csv', encoding='latin-1')

In [8]:
# # Preview the information of first 5 rows of dataset.

df_books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


### Read the data - BX-Ratings
To map byte values directly to the first 256 Unicode code points, use the "Latin-1" encoding. This is the closest equivalent Python 3 offers to the permissive Python 2 text handling model.

read only first 10000 rows otherwise, Out Of Memory error can occur.

In [9]:
# read dataset 
# read only 1st 10000 rows otherwise it might give us out of memory error

df = pd.read_csv('BX-Book-Ratings.csv', encoding='latin-1', nrows = 10000)

In [10]:
# Preview the information of first 5 rows of dataset.

df.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,155061224,5
2,276727,446520802,0
3,276729,052165615X,3
4,276729,521795028,6


In [33]:
# it shows basic Statistical details like percentile, mean, std deviation

df.describe()

,user_id_order,isbn_id,rating,user_id
count,8701.000000,8701.000000,8701.000000,8701.000000
mean,112.966326,3856.697736,1.921503,269561.008850
std,187.429148,2349.524848,3.399550,47850.105442
min,0.000000,0.000000,0.000000,2.000000
25%,3.000000,1783.000000,0.000000,277523.000000
50%,8.000000,3738.000000,0.000000,278418.000000
75%,135.000000,5900.000000,3.000000,278418.000000
max,827.000000,8050.000000,10.000000,278854.000000


### Merge the Dataframes 
   * for all the practical purposes, user is not required so ignore the dataframe df_user
   * merge only two datasets df and df_books

In [34]:
# inner join between two dataframes by common data or value = 'isbn'
df = pd.merge(df,df_books,on='isbn')
df.head()

,user_id_order,isbn_id,rating,book_title_x,book_author_x,year_of_publication_x,publisher_x,isbn,user_id,book_title_y,book_author_y,year_of_publication_y,publisher_y
0,0,0,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,034545104X,276725,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,1,1,5,Rites of Passage,Judith Rae,2001,Heinle,155061224,276726,Rites of Passage,Judith Rae,2001,Heinle
2,2,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,446520802,276727,The Notebook,Nicholas Sparks,1996,Warner Books
3,3,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,446520802,278418,The Notebook,Nicholas Sparks,1996,Warner Books
4,4,3,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,052165615X,276729,Help!: Level 1,Philip Prowse,1999,Cambridge University Press


### Get the number of unique users and books

Here we are using 'nunique()' function that returns the Series with the number of distinct observations over the requested axis.

In [13]:
n_users = df.user_id.nunique()
n_books = df.isbn.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Books: '+str(n_books))

Num. of Users: 828
Num of Books: 8051


### Convert ISBN variable to numeric numbers in order

In [14]:
# International Standard Book Number (ISBN)

isbn_list = df.isbn.unique()
print(" Length of isbn List:", len(isbn_list))

 Length of isbn List: 8051


* This is formatted as code

### Convert user_id variable to numeric type in order

In [15]:
def get_isbn_numeric_id(isbn):
    
    itemindex = np.where(isbn_list==isbn)
    
    return itemindex[0][0]

### Do the same for user_id , convert it into numeric and in order

In [16]:
userid_list = df.user_id.unique()
print(" Length of user_id List:", len(userid_list))

 Length of user_id List: 828


In [17]:
def get_user_id_numeric_id(user_id):
    
    itemindex = np.where(userid_list==user_id)
    
    return itemindex[0][0]

### Converting both user_id and isbn to ordered list i.e. from 0...n-1

In [18]:
df['user_id_order'] = df['user_id'].apply(get_user_id_numeric_id)

In [19]:
df['isbn_id'] = df['isbn'].apply(get_isbn_numeric_id)

In [20]:
df.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,user_id_order,isbn_id
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,0,0
1,276726,155061224,5,Rites of Passage,Judith Rae,2001,Heinle,1,1
2,276727,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,2,2
3,278418,446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,3,2
4,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,4,3


### Re-index columns to build matrix

In [21]:
new_col_order = ['user_id_order', 'isbn_id', 'rating', 'book_title', 'book_author','year_of_publication','publisher','isbn','user_id']

df = df.reindex(columns= new_col_order)

df.head()

,user_id_order,isbn_id,rating,book_title,book_author,year_of_publication,publisher,isbn,user_id
0,0,0,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,034545104X,276725
1,1,1,5,Rites of Passage,Judith Rae,2001,Heinle,155061224,276726
2,2,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,446520802,276727
3,3,2,0,The Notebook,Nicholas Sparks,1996,Warner Books,446520802,278418
4,4,3,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,052165615X,276729


### Train Test Split
   *Recommendation Systems are difficult to evaluate, but you will still learn how to evaluate them. In order to do this, you'll split your data into two sets. However, you won't do your classic X_train,X_test,y_train,y_test split. Instead, you can actually just segement the data into two sets of data:

#### Importing train_test_split model

In [22]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.30)

#### Approach: We will use Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering approaches can be divided into two main sections: user-item filtering and item-item filtering.

A user-item filtering will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked.

In contrast, item-item filtering will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items as input and outputs other items as recommendations.
  
  * Item-Item Collaborative Filtering: “Users who liked this item also liked …”
  * User-Item Collaborative Filtering: “Users who are similar to you also liked …”

In both cases, you create a user-book matrix which is built from the entire dataset.

Since you have split the data into testing and training, you will need to create two [828 x 8051] matrices (all users by all books). This is going to be a very large matrix

The training matrix contains 70% of the ratings and the testing matrix contains 30% of the ratings.

### Create two user-book matrix for training and testing
   * Indented block

In [23]:
#Create two user-book matrices, one for training and another for testing

train_data_matrix = np.zeros((n_users, n_books))

for line in train_data.itertuples():
    
    # update the ratings to each row
    
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_books))

for line in test_data.itertuples():

    test_data_matrix[line[1]-1, line[2]-1] = line[3]

### Import Pairwise Model

#### we can use the **pairwise_distances** function from sklearn to calculate the cosine similarity. Note, the output will range from 0 to 1 since the ratings are all positive.

In [24]:
from sklearn.metrics.pairwise import pairwise_distances

user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [25]:
user_similarity

array([[0., 1., 1., ..., 1., 1., 1.],
       [1., 0., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 1.],
       [1., 1., 1., ..., 1., 1., 0.]])

### Make predictions

In [26]:
def predict(ratings, similarity, type='user'):
    
    if type == 'user':
        
        mean_user_rating = ratings.mean(axis=1)
        
        #You use np.newaxis so that mean_user_rating has same format as ratings
        
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
        
    elif type == 'item':
        
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    
    return pred

In [27]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [28]:
item_prediction

array([[0.        , 0.00062112, 0.0006212 , ..., 0.00062156, 0.00062112,
        0.00062112],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05552795, 0.05552795, 0.05553485, ..., 0.05556762, 0.05552795,
        0.05552795],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Evaluation
There are many evaluation metrics, but one of the most popular metric used to evaluate accuracy of predicted ratings is Root Mean Squared Error (RMSE).

* Since, we only want to consider predicted ratings that are in the test dataset, we filter out all other elements in the prediction matrix with: prediction[ground_truth.nonzero()].

In [29]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# ground_truth = Actual

def rmse(prediction, ground_truth):
    
    prediction = prediction[ground_truth.nonzero()].flatten() 
    
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    
    return sqrt(mean_squared_error(prediction, ground_truth))

### Printing RMSE value for user based and item based collaborative filtering


In [30]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))

print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 7.6838704648025065
Item-based CF RMSE: 7.683469868874414


#### Both the approach yield almost same result

# End